In [1]:
import pandas as pd
import numpy as np
import datetime
import os

In [2]:
d=datetime.date.today()

In [76]:
date='2020-04-23'

In [127]:
distance_matrix = pd.read_csv(f'output/{date}/distance_matrix.csv',index_col=0)
distance_matrix

,don_id,rec_id,distance
0,don100,rec0,2555.437173
1,don100,rec1,2439.900358
2,don100,rec1000,2564.239624
3,don100,rec1001,1080.925456
4,don100,rec1004,164.486563
...,...,...,...
397686,don200,rec992,177.573542
397687,don200,rec993,2387.958603
397688,don200,rec994,1844.382298
397689,don200,rec996,12.287341


In [128]:
recipients = pd.read_csv(f'output/{date}/recipients.csv',index_col=0,parse_dates=['date'])
recipients

,rec_id,ppe,date,qty
0,rec0,faceShields,2020-04-02 16:27:00+00:00,5000.0
1,rec0,gowns,2020-04-02 16:27:00+00:00,1000.0
2,rec0,nitrileGloves,2020-04-02 16:27:00+00:00,10000.0
3,rec0,respirators,2020-04-02 16:27:00+00:00,10000.0
4,rec1,disinfectingWipes,2020-04-02 16:35:00+00:00,4.0
...,...,...,...,...
16624,rec998,respirators,2020-04-03 23:51:18+00:00,50.0
16625,rec998,thermometers,2020-04-03 23:51:18+00:00,2.0
16626,rec999,faceShields,2020-04-04 00:05:59+00:00,20.0
16627,rec999,rec_req_id,2020-04-04 00:05:59+00:00,999.0


In [129]:
donors = pd.read_csv(f'output/{date}/donors.csv',index_col=0,parse_dates=['date'])
donors

,don_id,ppe,date,qty
0,don100,nitrileGloves,2020-04-19 04:00:00+00:00,100.0
1,don101,surgicalMasks,2020-04-19 04:00:00+00:00,200.0
2,don102,respirators,2020-04-19 04:00:00+00:00,30.0
3,don103,respirators,2020-04-19 04:00:00+00:00,2.0
4,don104,faceShields,2020-04-19 04:00:00+00:00,20.0
...,...,...,...,...
241,don95,faceShields,2020-04-19 04:00:00+00:00,20.0
242,don96,surgicalMasks,2020-04-19 04:00:00+00:00,300.0
243,don97,faceShields,2020-04-19 04:00:00+00:00,100.0
244,don98,respirators,2020-04-19 04:00:00+00:00,1.0


# Take only one ppe

In [130]:
ppe='faceShields'

In [131]:
donors=donors[donors.ppe == ppe]
donors

,don_id,ppe,date,qty
4,don104,faceShields,2020-04-19 04:00:00+00:00,20.0
5,don105,faceShields,2020-04-19 04:00:00+00:00,200.0
7,don108,faceShields,2020-04-19 04:00:00+00:00,30.0
11,don115,faceShields,2020-04-19 04:00:00+00:00,25.0
24,don128,faceShields,2020-04-19 04:00:00+00:00,100.0
...,...,...,...,...
234,don88,faceShields,2020-04-19 04:00:00+00:00,50.0
235,don89,faceShields,2020-04-19 04:00:00+00:00,1000.0
237,don91,faceShields,2020-04-19 04:00:00+00:00,30.0
241,don95,faceShields,2020-04-19 04:00:00+00:00,20.0


In [132]:
recipients=recipients[recipients.ppe == ppe]
recipients

,rec_id,ppe,date,qty
0,rec0,faceShields,2020-04-02 16:27:00+00:00,5000.0
11,rec10,faceShields,2020-04-02 17:48:00+00:00,30.0
16,rec100,faceShields,2020-04-03 08:12:00+00:00,20.0
22,rec1000,faceShields,2020-04-04 00:26:00+00:00,1000.0
31,rec1001,faceShields,2020-04-04 00:35:24+00:00,10.0
...,...,...,...,...
16591,rec993,faceShields,2020-04-03 23:12:05+00:00,50.0
16598,rec994,faceShields,2020-04-03 23:14:56+00:00,10.0
16606,rec995,faceShields,2020-04-03 23:17:44+00:00,5.0
16620,rec998,faceShields,2020-04-03 23:51:18+00:00,10.0


# Proximity match

dummy_strategy(date,curdon,currec,curdistance_mat):

return dataframe of decisions (don_id,rec_id,ppe,qty)

In [133]:
result = pd.DataFrame(columns=['don_id','rec_id','ppe','qty'])

In [134]:
donors_ppe = donors.copy()
recipients_ppe=recipients.copy()
curdistance_mat=distance_matrix.copy()

In [135]:
donors_ppe

,don_id,ppe,date,qty
4,don104,faceShields,2020-04-19 04:00:00+00:00,20.0
5,don105,faceShields,2020-04-19 04:00:00+00:00,200.0
7,don108,faceShields,2020-04-19 04:00:00+00:00,30.0
11,don115,faceShields,2020-04-19 04:00:00+00:00,25.0
24,don128,faceShields,2020-04-19 04:00:00+00:00,100.0
...,...,...,...,...
234,don88,faceShields,2020-04-19 04:00:00+00:00,50.0
235,don89,faceShields,2020-04-19 04:00:00+00:00,1000.0
237,don91,faceShields,2020-04-19 04:00:00+00:00,30.0
241,don95,faceShields,2020-04-19 04:00:00+00:00,20.0


In [136]:
recipients_ppe

,rec_id,ppe,date,qty
0,rec0,faceShields,2020-04-02 16:27:00+00:00,5000.0
11,rec10,faceShields,2020-04-02 17:48:00+00:00,30.0
16,rec100,faceShields,2020-04-03 08:12:00+00:00,20.0
22,rec1000,faceShields,2020-04-04 00:26:00+00:00,1000.0
31,rec1001,faceShields,2020-04-04 00:35:24+00:00,10.0
...,...,...,...,...
16591,rec993,faceShields,2020-04-03 23:12:05+00:00,50.0
16598,rec994,faceShields,2020-04-03 23:14:56+00:00,10.0
16606,rec995,faceShields,2020-04-03 23:17:44+00:00,5.0
16620,rec998,faceShields,2020-04-03 23:51:18+00:00,10.0


In [137]:
dqty = drow.qty

In [138]:
for _, drow in donors_ppe.iterrows():
    # find the closest recipient
    dr = curdistance_mat[(curdistance_mat.don_id == drow.don_id)].merge(recipients_ppe,on='rec_id').sort_values('distance').iloc[0]
    dqty = drow.qty
    rqty = recipients_ppe.loc[recipients_ppe.rec_id == dr.rec_id,'qty'].values[0]
    qty = min(dqty,rqty)
    if qty == 0:
        print('qty is zero')
    if qty == rqty:
        recipients_ppe = recipients_ppe[recipients_ppe.rec_id !=  dr.rec_id] # remove recipient
    else:
        recipients_ppe.loc[recipients_ppe.rec_id == dr.rec_id,'qty'] -= qty #update recipient's qty
    result.loc[len(result),:] = [dr.don_id, dr.rec_id,ppe,qty]

    #print(dr)

break


modify

In [32]:
recipients.loc[1] = ['rec0','2020-04-02 16:27:00+00:00',ppe,6]

In [31]:
recipients.loc[6352] = ['rec0','2020-04-03 11:45:00+00:00',ppe,6]

In [33]:
recipients

,rec_id,date,ppe,qty
1,rec0,2020-04-02 16:27:00+00:00,respirators,6.0
2,rec0,2020-04-02 16:27:00+00:00,faceShields,5000.0
14,rec0,2020-04-02 16:27:00+00:00,gowns,1000.0
15,rec0,2020-04-02 16:27:00+00:00,nitrileGloves,10000.0
18,rec1,2020-04-02 16:35:00+00:00,respirators,9.0
...,...,...,...,...
6344,rec373,2020-04-03 11:45:00+00:00,thermometers,2.0
6347,rec373,2020-04-03 11:45:00+00:00,disinfectingWipes,7.0
6350,rec373,2020-04-03 11:45:00+00:00,handmadeMasks,15.0
6352,rec0,2020-04-03 11:45:00+00:00,respirators,6.0


In [34]:
dec = decisions.iloc[0]
dec

don_id                               don0
rec_id                               rec0
qty                                    20
remove_donor                         True
date            2020-04-03 11:46:00+00:00
ppe                           respirators
Name: 0, dtype: object

In [40]:
rem_qty_to_ship = dec.qty
rec = dec.rec_id

In [41]:
rec_df = recipients[(recipients.rec_id == rec)&(recipients.ppe == ppe)]
rec_df

,rec_id,date,ppe,qty
1,rec0,2020-04-02 16:27:00+00:00,respirators,6.0
6352,rec0,2020-04-03 11:45:00+00:00,respirators,6.0


In [42]:
requests_to_remove = []
for _,row in rec_df.iterrows():
    request_id = row.name
    request_qty = row.qty
    request_date = row.date
    if rem_qty_to_ship < request_qty:
        # TODO modify the row, but keep it, then exit the loop
        print('modify the row, but keep it, then exit the loop')
    else:
        # remove the row and update remo_qty_to_ship
        rem_qty_to_ship -= request_qty
        requests_to_remove.append(request_id)
        

In [44]:
rem_qty_to_ship

8.0

In [ ]:
requests_to_remove